In [1]:
import pandas as pd 
import requests
import numpy as np
import re
import warnings
from pandas import json_normalize

In [2]:
import os.path

file_exists = os.path.exists("inst_full.csv")

print(file_exists)

True


THE CELL BEOW CHECKS TO SEE IF YOU ALREADY HAVE THE DATA AND IF NOT SENDS GET REQUESTS TO THE API

NOTE - THIS DOES NOT CHECK TO ENSURE THE FILE IS CURRENT OR FREE OF ERRORS.  IF YOU WOULD LIKE TO UPDATE YOUR FILE OR YOU ARE CONCERNED THAT YOUR FILE HAS ERRORS, SIMPLY DELETE YOUR FILE AND A NEW, UPDATED FILE WILL BE CREATED.

1. KNOWLEDGE CHECK 2 - See the last 20 lines in the cell below (most of the code in the ELSE statement).  This cleans the data by checking for columns with mixed data types and converting the entire column to strings.  It converts the data type, but also displays an error to identify the affected columns.  This is a solution I found on stackoverflow that I wanted to implement in my code.

Knowledge Check 2 - Also see cells 4, 5, 7, 9, and 10

In [3]:

import pandas


if file_exists == False:
    print("FILE DOES NOT EXIST")

    response_API = requests.get('https://banks.data.fdic.gov/api/institutions?limit=10000&offset=0')
    #print(response_API.status_code)

    response_API2 = requests.get('https://banks.data.fdic.gov/api/institutions?limit=10000&offset=10000')
    #print(response_API2.status_code)

    response_API3 = requests.get('https://banks.data.fdic.gov/api/institutions?limit=10000&offset=20000')
    #print(response_API3.status_code)

    print("Get request sucessful from API")


    data = response_API.json()
    #print(type(data))

    data2 = response_API2.json()
    #print(type(data2))

    data3 = response_API3.json()
    #print(type(data3))



    institution_df = json_normalize(data['data'])
    #print(institution_df.head())
    #institution_df.to_csv('inst.csv')

    institution_df2 = json_normalize(data2['data'])
    #print(institution_df2.head())
    #institution_df2.to_csv('inst2.csv')

    institution_df3 = json_normalize(data3['data'])
    #print(institution_df3.head())
    #institution_df3.to_csv('inst3.csv')
    
    print("Data from API converted to json")


    institution_df = institution_df.reindex(sorted(institution_df.columns), axis=1)
    institution_df2 = institution_df2.reindex(sorted(institution_df2.columns), axis=1)
    institution_df3 = institution_df3.reindex(sorted(institution_df3.columns), axis=1)



    full_data = [institution_df, institution_df2, institution_df3]
    df = pd.concat(full_data)
    df.columns = df.columns.str.lstrip("data.")
    df.to_csv('inst_full.csv')
    
    print("New file created, shape should be (27816, 168)")
    print(df.shape)
    

else:
    print("FILE ALREADY EXISTS")

    myfile = 'inst_full.csv'
    target_type = str  # The desired output type

    with warnings.catch_warnings(record=True) as ws:
        warnings.simplefilter("always")

        mydata = pandas.read_csv(myfile, sep="|", header=None)
        print("Warnings raised:", ws)
        # We have an error on specific columns, try and load them as string
        for w in ws:
            s = str(w.message)
            print("Warning message:", s)
            match = re.search(r"Columns \(([0-9,]+)\) have mixed types\.", s)
            if match:
                columns = match.group(1).split(',') # Get columns as a list
                columns = [int(c) for c in columns]
                print("Applying %s dtype to columns:" % target_type, columns)
                mydata.iloc[:,columns] = mydata.iloc[:,columns].astype(target_type)

    df = pd.read_csv('inst_full.csv')
    

FILE ALREADY EXISTS
Warnings raised: []


C:\Users\mjlaw\AppData\Local\Temp\ipykernel_28264\3671726210.py:82: DtypeWarning: Columns (108,109,110,152) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('inst_full.csv')


2. Knowledge Check 2 - Removing unecessary rows

In [4]:
df = df.drop(df.iloc[:, 165:168],axis = 1)
df = df.drop(df.iloc[:, 141:156],axis = 1)
df = df.drop(df.iloc[:, 106:110],axis = 1)
df = df.drop(df.iloc[:, 22:31],axis = 1)

3. Knowledge Check 2 - Reformating the column names so they are all lower case and spaces are replaced with an underscore

In [5]:
#print(df.columns)
df.columns= df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')
#print(df.columns)

In [6]:
#To add banks with FDIC violations

absolute_path = os.path.abspath('')
relative_path = "ots-enforcement-order-listing.csv"
full_path = os.path.join(absolute_path, relative_path)
enforcment_df = pd.read_csv(full_path)
print(len(enforcment_df))
print(enforcment_df.head())

3547
   Record ID Docket Number     IID  \
0          1            29    29.0   
1          2          8327  8327.0   
2          3          3916  3916.0   
3          4           NaN     NaN   
4          5           NaN     NaN   

                                   Unnamed: 3 DOCASSOCIATION Unnamed: 5  \
0  http://www.occ.gov/static/ots/enforcement/          93296       .pdf   
1  http://www.occ.gov/static/ots/enforcement/          93221       .pdf   
2  http://www.occ.gov/static/ots/enforcement/          93233       .pdf   
3  http://www.occ.gov/static/ots/enforcement/          93799       .pdf   
4  http://www.occ.gov/static/ots/enforcement/          94981       .pdf   

                                          Unnamed: 6  \
0  http://www.occ.gov/static/ots/enforcement/9329...   
1  http://www.occ.gov/static/ots/enforcement/9322...   
2  http://www.occ.gov/static/ots/enforcement/9323...   
3  http://www.occ.gov/static/ots/enforcement/9379...   
4  http://www.occ.gov/static/ots/en

Knowledge Check 2 - making all column names lower case and replacing spaces with underscores

In [7]:
#print(enforcment_df.columns)
enforcment_df.columns= enforcment_df.columns.str.lower()
enforcment_df.columns = enforcment_df.columns.str.replace(' ', '_')
#print(enforcment_df.columns)

Index(['Record ID', 'Docket Number', 'IID', 'Unnamed: 3', 'DOCASSOCIATION',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Link to Enforcement Action',
       'Order Number', 'Institution Name', 'First Name', 'Last Name', 'City',
       'State', 'OTS Region', 'Issue Date', 'Type Code', 'Type Description',
       'Type of Order', 'Notes'],
      dtype='object')
Index(['record_id', 'docket_number', 'iid', 'unnamed:_3', 'docassociation',
       'unnamed:_5', 'unnamed:_6', 'unnamed:_7', 'link_to_enforcement_action',
       'order_number', 'institution_name', 'first_name', 'last_name', 'city',
       'state', 'ots_region', 'issue_date', 'type_code', 'type_description',
       'type_of_order', 'notes'],
      dtype='object')


In [8]:
#for d in enforcment_df['docket_number'].astype(str):
#    if enforcment_df['docket_number'].str.startswith('H'):
#        enforcment_df['docket_number'].str.replace(' ', '')

4. Knowledge Check 2 - Removes improperly formatted docket numbers

In [9]:
enforcment_df['docket_number'] = np.where(enforcment_df['docket_number'].str.startswith('H'), '', enforcment_df['docket_number'])

5. Knowldge Check 2 - Removes rows with no docket number

In [10]:
#create a new df so the old df is unaltered

#sorts the new df by docket number
enf_df1 = enforcment_df.sort_values('docket_number')

#converts emtpy strings to NaN and then drops rows that do not have a Docket Number
enf_df1['docket_number'].replace('', np.nan, inplace=True)
enf_df1.dropna(subset=['docket_number'], inplace=True)

#Gives the size and first columns of the new_enf_df
print(len(enf_df1.sort_values('docket_number')))
print(enf_df1.head(5))

2029
      record_id docket_number      iid  \
1391       1463           100    100.0   
2907       3018           100    100.0   
1268       1337           100    100.0   
538         551         10100  10100.0   
2382       2478          1016   1016.0   

                                      unnamed:_3 docassociation unnamed:_5  \
1391  http://www.occ.gov/static/ots/enforcement/          94571       .pdf   
2907  http://www.occ.gov/static/ots/enforcement/          97102       .pdf   
1268  http://www.occ.gov/static/ots/enforcement/          93646       .pdf   
538   http://www.occ.gov/static/ots/enforcement/          93574       .pdf   
2382  http://www.occ.gov/static/ots/enforcement/          96124       .pdf   

                                             unnamed:_6  \
1391  http://www.occ.gov/static/ots/enforcement/9457...   
2907  http://www.occ.gov/static/ots/enforcement/9710...   
1268  http://www.occ.gov/static/ots/enforcement/9364...   
538   http://www.occ.gov/static/ots/e

In [11]:
#drops duplicates and adds 2 new colums to the new_enf_df

short_enf_df = enf_df1.drop_duplicates(subset=['docket_number'], keep='last')
#short_enf_df = short_enf_df.reindex(columns = new_enf_df.columns.tolist() + ['Number_of_Violations','Unique_Violations'])

#Gives the size and first columns of the short_enf_df
#print(len(short_enf_df))
#print(short_enf_df.head())

In [12]:
violation_count_df = enf_df1.pivot_table(index = ['docket_number'], aggfunc ='size')
vc_df = violation_count_df.reset_index()
vc_df.columns = ['docket_number', 'total_violations']
#print(vc_df)

In [13]:
unique_violation_count_df = enf_df1.pivot_table(index = ['docket_number', "issue_date"], aggfunc ='size')
uvc_df = unique_violation_count_df.reset_index()
uvc_df.columns = ['docket_number', 'issue_date', 'violations']
#print(uvc_df)

In [14]:
final_uvc_df = uvc_df.pivot_table(index = ['docket_number'], aggfunc ='size')
fvc_df = violation_count_df.reset_index()
fvc_df.columns = ['docket_number', 'unique_violations']
#print(fvc_df)

In [15]:
violation_stats_df = vc_df.merge(fvc_df)
#print(violation_stats_df)

In [16]:
final_enf_df = short_enf_df.merge(violation_stats_df)
#print(final_enf_df.head)

In [17]:
df['docket'] = df['docket'].astype(str)
final_df = df.merge(final_enf_df, how='left', left_on='docket', right_on='docket_number')
print(final_df.head)

<bound method NDFrame.head of        unnamed:_0  active              address address2     asset bkclass  \
0               0       0  8 Washington Street      NaN       NaN      SM   
1               1       0    201 N.E. A Street      NaN  712714.0      SM   
2               2       0  1701 Warwood Avenue      NaN  338215.0       N   
3               3       0   1 East Main Street      NaN  126549.0      SM   
4               4       1        86 E Water St      NaN  144049.0      NM   
...           ...     ...                  ...      ...       ...     ...   
27811        7811       0     3425 Main Street      NaN       NaN      NM   
27812        7812       0          Main Street      NaN   62371.0      NM   
27813        7813       0       333 Penco Road      NaN  178789.0      NM   
27814        7814       1  105 E Washington St      NaN  454657.0      NM   
27815        7815       1          2 W Main St      NaN  200966.0      NM   

       callform   cb                         